In [1]:
import requests
import shutil
from bs4 import BeautifulSoup
import urllib.request
import shutil
import os
from ebooklib import epub
import time#
import urllib

# mobi格式简介 https://www.cnblogs.com/buptzym/p/5249662.html

s = requests.Session()
s.headers = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36"
}


epubTOC = []
book = epub.EpubBook()

# add copyright description
copyright = epub.EpubHtml(title="版权声明", file_name="copyright.html")
copyright.content = """<h1>版权声明</h1>
<p>本工具目的是将免费网络在线小说转换成方便kindle用户阅读的mobi电子书, 作品版权归原作者或网站所有, 请不要将该工具用于非法用途。</p>
<p>GitHub: https://github.com/fondoger/qidian2mobi</p>
"""
book.add_item(copyright)
epubTOC.append(epub.Link("copyright.html", "版权声明", "intro"))

In [3]:
def handle_url(url):
    # handle urls like `//example.com`
    if url[:2] == '//':
        return "http:" + url
    return url

def download_image(url, path):
    url = handle_url(url)
    urllib.request.urlretrieve(url, path)
    print("success saved book cover image:", url)

chapters_count = 0
def handle_chapter(chapter: BeautifulSoup) -> epub.EpubHtml:
    global chapters_count
    # chapter_name = chapter.get_text()
    chapter_name = chapter.find('a').get_text()
    chapter_url = chapter.find('a')['href']
    print(chapters_count, chapter_name, handle_url(chapter_url))
    r = s.get(handle_url("https://www.uukanshu.com"+chapter_url))
    soup = BeautifulSoup(r.content, features="html.parser")
    # content = soup.find('div', {'class': 'read-content'})
    content = soup.find(id="contentbox")
    chapters_count += 1
    c = epub.EpubHtml(title=chapter_name,
            file_name="chapter_%d.html" % chapters_count)
    c.set_content("<h1>" + chapter_name + "</h1>" + str(content))
    book.add_item(c)
    # time.sleep(0 if random.randint(0, 10) < 8 else 1)
    return c

def handle_volume(volume: BeautifulSoup):
    volume_name = volume.find('h3').get_text()[56:]
    print("handling volume: " + volume_name)
    chapters = volume.find_all('li')
    global temp
    epub_chapters = []
    for chapter in chapters:
        c = handle_chapter(chapter)
        epub_chapters.append(c)
    epubTOC.append((epub.Section(volume_name), epub_chapters))

def handle_list(theList: BeautifulSoup):
    epc = []
    lastVol = ""
    for item in theList:
        if item.has_attr("class") and item["class"][0] == "volume": # and item.get_text() != lastVol:
            
            if epc:
                epubTOC.append((epub.Section(lastVol), epc))
            print("handling volume: " + item.get_text())
            lastVol = item.get_text()
            epc = []
        else:
            chap = handle_chapter( item)
            epc.append(chap)
    if epc:
        epubTOC.append((epub.Section(lastVol), epc))

In [4]:
UU_URL = "https://www.uukanshu.com/b/69778/"
TITLE = "凡人修仙之仙界篇"
AUTHOR = "忘语"
"unicode-escape"

'unicode-escape'

In [7]:
urllib.urlopen("https://www.uukanshu.com/b/69778/").read()

AttributeError: module 'urllib' has no attribute 'urlopen'

In [12]:
# r = s.get(UU_URL)
r = urllib.request.urlopen("https://www.uukanshu.com/b/69778/").read()
# r.encoding = 'GBK'
# soup = BeautifulSoup(r.content, features="html.parser")
soup = BeautifulSoup(r.decode("GBK"), features="html.parser")
book_cover_src = soup.find('a', {'class': 'bookImg'}).find('img')['src']
# print(f"book cover src {book_cover_src}")
book_title = soup.find('a', {'class': 'bookImg'}).find('img')['alt']

print("book cover src", book_cover_src, "book title", book_title)

book cover src //img.uukanshu.com/fengmian/2017/11/636451526950602517.jpg book title 凡人修仙之仙界篇


In [30]:
_jieshao = soup.find_all("dd", {"class":"jieshao_content"})[0]

_jieshao

<dd class="jieshao_content">
<h1><a href="/b/69778/" title="·²ÈËÐÞÏÉÖ®ÏÉ½çÆª×îÐÂÕÂ½Ú">·²ÈËÐÞÏÉÖ®ÏÉ½çÆª×îÐÂÕÂ½Ú</a></h1>
<h2>·²ÈËÐÞÏÉÖ®ÏÉ½çÆª×÷Õß£º<a href="/search.aspx?t=1&amp;key=%cd%fc%d3%ef" target="_blank">ÍüÓï</a></h2>
<h3>·²ÈËÐÞÏÉÖ®ÏÉ½çÆª¼ò½é£º www.uukanshu.com ·²ÈËÐÞÏÉ£¬·çÔÆÔÙÆð<br/>¡¡¡¡Ê±¿Õ´©Ëó£¬ÂÖ»ØÄæ×ª<br/>¡¡¡¡½ðÏÉÌ«ÒÒ£¬´óÂÞµÀ×æ<br/>¡¡¡¡ÈýÇ§´óµÀ£¬·¨ÔòÖÁ×ð<br/>¡¡¡¡¡¶·²ÈËÐÞÏÉ´«¡·ÏÉ½çÆª£¬Ò»¸öº«Á¢ß³ßåÏÉ½çµÄ¹ÊÊÂ£¬Ò»¸ö·²ÈËÐ¡×ÓÐÞÏÉµÄ²»Ãð´«Ëµ¡£ https://www.uukanshu.com              
              <br>
                £­£­£­£­£­£­£­£­£­£­£­£­£­£­£­£­£­£­£­£­£­£­£­£­£­£­£­£­£­£­£­£­£­£­£­£­£­
              </br></h3>
<div class="zuixin">
<b>·²ÈËÐÞÏÉÖ®ÏÉ½çÆª×îÐÂÕÂ½Ú</b>£º<a href="/b/69778/226130.html" target="_blank">ÐÂÊé¡¶´óÃÎÖ÷¡·ÉÏ¼Ü¸ÐÑÔ</a>
<span class="ma_ico" id="mobileread" onmouseout="this.className = this.className.replace(' ma_ico_hover','')" onmouseover='if(this.className.search(" ma_ico_hover")==-1){this.className+=" ma_ico_hover";ShowQRCode(69778,0,1);}'>
<span class="ma_box

In [31]:
author = _jieshao.find('h2').find("a").get_text()

In [32]:
theList = soup.find(id="chapterList")
# theList.find_all("li")
handle_list(reversed(list(theList.find_all("li"))))

handling volume: ÕýÎÄ¾í
0 µÚ1ÕÂ ºüÅ® /b/69778/13861.html
1 µÚ2ÕÂ Ê¯Í·¸ç¸ç /b/69778/13862.html
2 µÚ3ÕÂ Ô¶È¥ /b/69778/14437.html
3 µÚ4ÕÂ ÏàÒÀ /b/69778/16007.html
4 µÚ5ÕÂ ÂíÊÞ /b/69778/16757.html
5 µÚ6ÕÂ °×ÅÛÉÙÄê /b/69778/17221.html
6 µÚ7ÕÂ Óà¸® /b/69778/17681.html
7 µÚ8ÕÂ °×Ê¯ÕæÈË /b/69778/21166.html
8 µÚ9ÕÂ ·¨Õó /b/69778/24237.html
9 µÚ10ÕÂ Î£»ú /b/69778/25631.html
10 µÚ11ÕÂ ËÕÐÑ /b/69778/26690.html
11 µÚ12ÕÂ ÎÒÊÇº«Á¢ /b/69778/27624.html
12 µÚ13ÕÂ Ä§¹â /b/69778/28499.html
13 µÚ14ÕÂ Ôâ±ä /b/69778/29694.html


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


14 µÚ15ÕÂ ÁîÅÆ /b/69778/53231.html
15 µÚ16ÕÂ ¾øÍû /b/69778/53914.html
16 µÚ17ÕÂ Ãð2ÐÞ /b/69778/54542.html


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


17 µÚ18ÕÂ ÇëÇó /b/69778/55327.html
18 µÚ19ÕÂ »¯ÉñÖ¶Ëï /b/69778/55964.html
19 µÚ20ÕÂ ÑûÇë /b/69778/56616.html
20 µÚ21ÕÂ ÔªÓ¤Òì±ä /b/69778/57267.html
21 µÚ22ÕÂ ·çÔÆË«É· /b/69778/57993.html
22 µÚ23ÕÂ Õù¶· /b/69778/58708.html
23 µÚ24ÕÂ ÄëÑ¹ /b/69778/59342.html
24 µÚ25ÕÂ Â½ÑÂ /b/69778/60028.html
25 µÚ26ÕÂ µÐ×ÙÔÙÏÖ /b/69778/60659.html
26 µÚ27ÕÂ µ£5ÔÀ£¬Ñ¹5¹í /b/69778/61308.html


...

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


229 µÚ224ÕÂ ÒÆ»ö½­¶« /b/69778/120821.html
230 µÚ225ÕÂ ÓÖ¼ûËï¿Ë /b/69778/121123.html
231 µÚ226ÕÂ ÏÉË× /b/69778/121187.html
232 µÚ227ÕÂ 3Â· /b/69778/121760.html
233 µÚ228ÕÂ ²î¾à /b/69778/121823.html
234 µÚ229ÕÂ ±»À§ /b/69778/122096.html
235 µÚ230ÕÂ »°¶à /b/69778/122143.html
236 µÚ231ÕÂ ÖØË®Ö®Íþ /b/69778/122419.html
237 µÚ232ÕÂ ÊÍÒÉ /b/69778/122474.html
238 µÚ233ÕÂ Ö²¶¹±ø /b/69778/122732.html
239 µÚ234ÕÂ ºï¶ù¾Æ /b/69778/122796.html
240 µÚ235ÕÂ ºïÍõÏ×±¦ /b/69778/123036.html
241 ·²ÈËµÚ9½ìÊéÆÀÈü¡°ÃÎÏë²»ËÀ¡±±­ÕýÊ½¿ªÈü /b/69778/123093.html
242 µÚ236ÕÂ ³õÉæÏÉµ¤ /b/69778/123104.html
243 µÚ237ÕÂ ·¨ÔòÁ¶µ¤ /b/69778/123338.html
244 µÚ238ÕÂ ÄÒÖÐÐßÉ¬ /b/69778/123417.html
245 µÚ239ÕÂ ¿ñÈË /b/69778/123662.html
246 µÚ240ÕÂ ½ðÏÉÒéÊÂ /b/69778/123732.html
247 µÚ241ÕÂ ÷ë9´«Ñ¶ /b/69778/123984.html
248 µÚ242ÕÂ ÈÛ½ðÁ¶½£ /b/69778/124064.html
249 µÚ243ÕÂ ¼«º® /b/69778/124304.html
250 µÚ244ÕÂ ÅÐ¶Ï /b/69778/124360.html
251 µÚ245ÕÂ Èë¹È /b/69778/124620.html


: 

: 

: 

In [ ]:

theList = soup.find(id="chapterList")
handle_list(reversed(list(theList.descendants)))

# set title
book.set_title(book_title)
# set author
book.add_author(book_author)
# set cover
download_image(book_cover_src, "book_cover.jpg")
book.set_cover('cover.jpg', open('book_cover.jpg', 'rb').read())
# set book language
book.set_language('zh_Hans')
# set book's TOC
book.toc = epubTOC
# add navigation files
book.add_item(epub.EpubNcx())
book.add_item(epub.EpubNav())

# save book
epub.write_epub('book.epub', book)
print(book_title + "-" + book_author, end=" ")
print("successfully saved to book.epub")

print("you can use kindlegen tool to convert epub to mobi")
print("for example: `bin/kindlegen book.epub`")
print("(choose executable file in bin/ directory depending on your operating system.)")